In [3]:
!pip install torchtext

  Using cached torchtext-0.15.2-cp310-cp310-win_amd64.whl (1.9 MB)


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

c:\Users\galle\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torchtext.datasets import IMDB

# Define the model
class SentimentAnalysisModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, text):
        embedded = self.embedding(text)
        output, _ = self.rnn(embedded)
        output = self.fc(output)
        return output

# Padding function
def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)

    return xx_pad, torch.Tensor(yy), x_lens

# Load data
train_iter, test_iter = IMDB()

# Convert to list
train_dataset = list(train_iter)
test_dataset = list(test_iter)

print(train_dataset[5000])

# Split data
train_data, valid_data = train_test_split(train_dataset, test_size=0.1)

# Define the DataLoader
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, collate_fn=pad_collate)
valid_loader = DataLoader(valid_data, batch_size=64, shuffle=True, collate_fn=pad_collate)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, collate_fn=pad_collate)

# Define the model
model = SentimentAnalysisModel(25000, 100, 256, 1)

# Define the loss function and the optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

# Training
for epoch in range(5):
    for i, (text, labels, _) in enumerate(train_loader):
        optimizer.zero_grad()
        predictions = model(text).squeeze(1)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()


(1, 'I used to be an avid viewer until I personally spent long cold hours helping build a home for the White Family, only to be sickened to see the house a year later. All of the beautiful rock landscaping has been removed, the gorgeous rock sidewalk and front fountain have been removed, all the pine trees and pecan trees in the front have been cut down, sprinkler system has been ripped out. It now looks like a disaster area. They don\'t even live there any more... they live "in town" and come out only for the weekend. It sickens me to think of all the hours that the great people of Oklahoma donated to these people and to see the result. The story that we all saw on TV wasn\'t completely the truth... don\'t believe every thing you see and hear.')


TypeError: object of type 'int' has no len()